# **Trabalho HoG (Histogram of Gradients) - Ciência da Computação - Daiane Babicz**

**Instalando dependências**

In [7]:
!pip install tensorflow
!pip install opencv-python tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Importando as bibliotecas necessárias**

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')
import cv2
import numpy as np
import os
from skimage.feature import hog
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Função para carregar as imagens e extrair os descritores HoG**

In [9]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for label, subfolder in enumerate(os.listdir(folder)):
        for filename in os.listdir(os.path.join(folder, subfolder)):
            img = cv2.imread(os.path.join(folder, subfolder, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (64, 128))
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_images_from_folder('/content/gdrive/MyDrive/Visão Computacional/Trabalho HoG/fruits-dataset')

**Reshape das imagens para o formato (n_samples, img_height, img_width, n_channels)**

In [10]:
images = images.reshape(-1, 128, 64, 1)

**Dividindo o conjunto de dados em treino e teste**

In [11]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=4000)

**Convertendo os labels para o formato categórico**

In [12]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

**Definindo a arquitetura do CNN**

In [13]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax')) 

**Compilando o modelo**

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Treinando o modelo**

In [15]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)


Epoch 1/10
58/58 [==============================] - 31s 524ms/step - loss: 30.5892 - accuracy: 0.2290 - val_loss: 1.5676 - val_accuracy: 0.2629
Epoch 2/10
58/58 [==============================] - 30s 517ms/step - loss: 1.4698 - accuracy: 0.3600 - val_loss: 1.5680 - val_accuracy: 0.3335
Epoch 3/10
58/58 [==============================] - 30s 513ms/step - loss: 1.1448 - accuracy: 0.5295 - val_loss: 1.7660 - val_accuracy: 0.3686
Epoch 4/10
58/58 [==============================] - 29s 506ms/step - loss: 0.7443 - accuracy: 0.7211 - val_loss: 2.2260 - val_accuracy: 0.3845
Epoch 5/10
58/58 [==============================] - 30s 527ms/step - loss: 0.4333 - accuracy: 0.8499 - val_loss: 2.5981 - val_accuracy: 0.4042
Epoch 6/10
58/58 [==============================] - 29s 507ms/step - loss: 0.2782 - accuracy: 0.9127 - val_loss: 2.9133 - val_accuracy: 0.4252
Epoch 7/10
58/58 [==============================] - 31s 530ms/step - loss: 0.1820 - accuracy: 0.9460 - val_loss: 3.6871 - val_accuracy: 0.422

**Avaliando o modelo**

In [21]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 4.080855846405029
Test accuracy: 0.4334818720817566
